## Guider mode quick look
This is intended to unpack the rawStamps from guider mode FITS files and 
give a quick look of a single stamp for one CCD. \
Craig Lage - 16-Apr-25

In [ ]:
import os
import shlex, subprocess
import matplotlib.pyplot as plt
from lsst.summit.utils.plotting import plot
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath
import matplotlib.colors as colors
from lsst.summit.utils import getQuantiles
import scipy.ndimage as ndi

# Get the main header and the information it contains

In [ ]:
def getMainHeaderInfo(hdu_list):
    hdr0 = hdu_list[0].header
    roiCol = hdr0['ROICOL']
    roiRow = hdr0['ROIROW']
    roiCols = hdr0['ROICOLS']
    roiRows = hdr0['ROIROWS']
    try:
        roiUnder = hdr0['ROIUNDRC']
    except:
        roiUnder = 6
    nStamps = hdr0['N_STAMPS']
    
    # Set the xor value - Guider CCDs are different from science CCDs
    if raft in ['R00', 'R04', 'R40', 'R44']:
        # Guider rafts
        xor = 0x20000
    else:
        # Science rafts
        xor = 0x1ffff
    return [roiRow, roiCol, roiRows, roiCols, roiUnder, nStamps, xor]

# Now define the code to unpack the rawStamps:

In [ ]:
def unpackStamps(hduNum):
    data = np.array(hdu_list[hduNum].data[0]).astype('>u4')[0]
    data.byteswap(inplace=True)
    totalCols = roiCols + roiUnder
    size = roiRows * totalCols
    out = np.zeros([16, size], dtype=int)
    image_out = np.zeros([16, roiRows, roiCols], dtype=int)
    for n in range(size):
        # Get 9 32 bit words of data
        res = ''
        for i in range(9):
            d = data[(size - n) * 9 - i - 1]
            d = format(d, '#034b')
            d = d.split('b')[1]
            res += d
        # Now extract 16 18 bit words from the data
        for i in range(16):
            bin_value = res[i * 18:(i + 1) * 18]
            int_value = int(bin_value, 2)
            final_value = int_value ^ xor
            out[i,n] = final_value  
    for i in range(16):
        reshaped = out[i,:].reshape(roiRows, totalCols)
        image_out[i,:,:] = np.flipud(np.fliplr(reshaped[:,0:roiCols]))
    return image_out

# Build a movie of a single stamp

In [ ]:
dayObs = 20250414
seqNum = 660
seg = 12
expId = int(f"{dayObs}{seqNum:05d}")
raft = 'R00'
ccd = 'SG1'
filename = f"s3://embargo@rubin-summit/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"

rp = ResourcePath(filename)
with rp.open(mode="rb") as f:
    hdu_list = fits.open(f)
[roiRow, roiCol, roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
detName = f"{raft}_{ccd}"

dirName = f"/home/c/cslage/u/Guider_Mode/LSSTCam_movie_{expId}_{raft}_{ccd}_{seg}"
%mkdir -p {dirName}
movieName = f"Guider__{expId}_{raft}_{ccd}_{seg}.mp4"
print(movieName)
# Build the individual frames

fig = plt.figure(figsize=(10,10))
xcom = []
ycom = []
for n in range(1, nStamps):    
    hduNum = 2 * n + 1
    hdrn = hdu_list[hduNum].header
    timestamp = hdrn['STMPTIME']
    image_out = unpackStamps(hduNum)
    roiarr = image_out[seg]
    com = ndi.center_of_mass((roiarr-np.median(roiarr)))
    xcom.append(com[0] * 0.2)
    ycom.append(com[1] * 0.2)
    plt.suptitle(f"Guider mode {expId} {raft}_{ccd}_{seg}, \n Frame {n+1} {timestamp}", fontsize=18) 
    plt.imshow(roiarr, interpolation='nearest', origin='lower', cmap='Greys')
    plt.savefig(f"{dirName}/Frame_{n:03d}.png")
    plt.clf()
    if n % 10 == 0:
        print(f"Finished frame {n}")
print("Done building frames")    



In [ ]:
print(f"\033[1mThe movie name will be: {dirName}/{movieName}\033[0m")

command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/{movieName}"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()